In [15]:
from Bio.Align import substitution_matrices
from Bio import Align
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
import pandas as pd
from Bio import SeqIO
import json
import os
import numpy as np
import datetime
import re


# mbs_consensus_names: pre kazdy virus nazov(id)
mbs_consensus_names = {
                        'h1n1': 'run230320_UVZ_BA_23-vsp-1337_A__WISCONSIN__588__2019__H1N1_',
                        'h3n2': 'run230320_UVZ_BA_23-vsp-2129_A__DARWIN__6__2021__H3N2_',
                        'rsv_b': 'run230320_UVZ_BA_23-vsp-902_hRSV__B__Australia__VIC-RCH056__2019:EPI_ISL_1653999:2019-03-04', 
                        'sars_cov_2_122': 'Consensus_uvzsr-BA_24_00001-G12-BA_24_00000122_threshold_0.5_quality_20',  # mozno bude treba zmenit hlavicku v consenzus sekvencii pre sars_cov_2 a zjednotit s generovanymi vysledkami
                        'sars_cov_2_139': 'Consensus_uvzsr-BA_24_00002-A09-BA_24_00000139_threshold_0.5_quality_20',
                        'sars_cov_2_4964': 'run230710_UVZ_BA_23-vsp-4964_NC_045512.2',
                        'rsv_a': 'run230320_UVZ_BA_23-vsp-1658_hRSV__A__England__397__2017:EPI_ISL_412866:2017-01-01'
                       }

samples_names = {
                 'h1n1': 'run230320_UVZ_BA_23-vsp-1337_',
                  'h3n2': 'run230320_UVZ_BA_23-vsp-2129_',
                  'rsv_b': 'run230320_UVZ_BA_23-vsp-902_', 
                  'sars_cov_2_139': 'uvzsr-BA_24_00002-A09-BA_24_00000139_',
                  'sars_cov_2_122': 'uvzsr-BA_24_00001-G12-BA_24_00000122_',
                  'sars_cov_2_4964': 'run230710_UVZ_BA_23-vsp-4964_',
                  'rsv_a': 'run230320_UVZ_BA_23-vsp-1658_'
                 }

virus_reference = {
                  'h1n1': 'h1n1_2019',
                   'h3n2': 'h3n2_2021', 
                   'rsv_b': 'rsv_b_2019', 
                   'sars_cov_2_139': 'sars_cov_2',
                   'sars_cov_2_122': 'sars_cov_2',
                   'sars_cov_2_4964': 'sars_cov_2',
                   'rsv_a': 'rsv_a_2017'
}

parameteres_df = pd.read_csv('./data/parameters_set_trimm_vc.tsv', sep='\t')

mbs_consensus_path_dict = {
    'h1n1': './consensus_baseline/1337_consensus-h1n1_2019-wgs.fa',
    'h3n2': './consensus_baseline/2129_consensus-h3n2_2021-wgs.fa',
    'rsv_b': './consensus_baseline/902_consensus-rsv_b_2019-wgs.fa',
    'rsv_a': './consensus_baseline/1658_consensus-rsv_a_2017-wgs.fa',
    'sars_cov_2_139': './consensus_baseline/139_consensus-sars_cov_2-wgs.fa',
    'sars_cov_2_122': './consensus_baseline/122_consensus-sars_cov_2-wgs.fa',
    'sars_cov_2_4964': './consensus_baseline/4964_consensus-sars_cov_2-wgs-respiratory.fa'
}


def set_scores_PairwiseAligner_rsv_b(seqA, seqB, match_score, mismatch_score, open_gap_score, extend_gap_score):

    aligner = Align.PairwiseAligner()
    aligner.mode = "global" 
    aligner.match_score = match_score
    aligner.mismatch_score = mismatch_score
    aligner.open_gap_score = open_gap_score
    aligner.extend_gap_score = extend_gap_score

    alignment = aligner.align(seqA, seqB)
    alignments = alignment
    
    # vyber znacky zo zarovania
    align_signs = str(alignment[0]).split('\n')[1]
    n_count = str(seqB).count('N')

    return {'score': alignment.score, "matches": align_signs.count('|'), 
            "gaps":  align_signs.count('-'), "mismatches":  align_signs.count('.'), "N_count": n_count}


def get_scores_for_sample(virus, sample_consensus, mbs_consensus, mbs_consensus_name, 
                          match_score, mismatch_score, open_gap_score, extend_gap_score):
    
    score_metrics_dict = {}

    score_all_fragments = 0
    match_all_fragments = 0
    gaps_all_fragments = 0
    mismatch_all_fragments = 0
    n_count_all_fragments = 0

    print('sample_consensus: ', sample_consensus.keys())

    # fragmenty pre vzorku
    for sample_fragment in sample_consensus.keys():
        
        print('sample_fragment: ', sample_fragment)
        fragment = sample_fragment.split('_')[-1] # skratka fragmentu genu
        print('fragment: ', fragment)

        # sekvencia (fragmentu) vzorky
        print('sample_consensus[sample_fragment]: ', sample_consensus[sample_fragment])
        sample_consensus_seq = sample_consensus[sample_fragment].seq 
        
        # sekvencia (fragmentu) mbs
        if virus == 'rsv_a' or virus == 'rsv_b' or virus == 'sars_cov_2_122' or virus == 'sars_cov_2_139' or virus == 'sars_cov_2_4964':
            
            print('mbs_consensus[mbs_consensus_names[virus]]: ', mbs_consensus[mbs_consensus_names[virus]])
            mbs_consensus_seq = mbs_consensus[mbs_consensus_names[virus]].seq
        
        elif virus == 'h1n1':
            if 'H1N1' in sample_fragment:
                mbs_consensus_fragment = mbs_consensus_name + fragment
                
            else:
                mbs_consensus_fragment = 'run230320_UVZ_BA_23-vsp-1337_A__WISCONSIN__588__2019__H1N1_PB2' 

            print('mbs_consensus_fragment', mbs_consensus_fragment)
            print(mbs_consensus[mbs_consensus_fragment])
            mbs_consensus_seq = mbs_consensus[mbs_consensus_fragment].seq

        else: # virus == 'h3n2':
            if 'H3N2' in sample_fragment:
                mbs_consensus_fragment = mbs_consensus_name + fragment
                
            else:
                mbs_consensus_fragment = 'run230320_UVZ_BA_23-vsp-2129_A__DARWIN__6__2021__H3N2_PB2' 

            print('mbs_consensus_fragment', mbs_consensus_fragment)
            print(mbs_consensus[mbs_consensus_fragment])
            mbs_consensus_seq = mbs_consensus[mbs_consensus_fragment].seq
        
        # prorovnanie consenzov (po fragmentoch), out: dict{score, gap, mismatch}; ulozenie do dict na poziciu fragmentu
        score_metrics_dict[fragment]  = set_scores_PairwiseAligner_rsv_b(mbs_consensus_seq, sample_consensus_seq, match_score, mismatch_score, open_gap_score, extend_gap_score)

        # cout scores for all fragments
        score_all_fragments += score_metrics_dict[fragment]['score']
        match_all_fragments += score_metrics_dict[fragment]['matches']
        gaps_all_fragments += score_metrics_dict[fragment]['gaps']
        mismatch_all_fragments += score_metrics_dict[fragment]['mismatches']
        n_count_all_fragments += score_metrics_dict[fragment]['N_count']

    # pridaj parameter pre vsetky fragmenty (score = count cez vsetky fragmenty)
    score_metrics_dict['all_fragments'] = {'score': score_all_fragments, 'matches': match_all_fragments, 'gaps': gaps_all_fragments, 
                                           'mismatches': mismatch_all_fragments, 'N_count':  n_count_all_fragments}
    
    print(score_metrics_dict)

    return score_metrics_dict


def get_samples_score_metrics_dict(virus, match_score, mismatch_score, open_gap_score, extend_gap_score):

    samples_score_metrics_dict = {}

    consensus_baseline_path = mbs_consensus_path_dict[virus]

    # nazov vzorky
    for sample_id in [samples_names[virus]]:
        
        print(sample_id)
        
        samples_score_metrics_parameter_dict = {}

        # cez vyber nastaveni respiracnej pipeliny
        for parameter in list(parameteres_df['parameters']):
            
            sample_name = sample_id + parameter

            consensus_sample_path = "./data/report_pollux/" + sample_name + "/_summary/consensus-" + virus_reference[virus] + "-wgs.fa"

            # print(consensus_sample_path)
            
            if os.path.exists(consensus_sample_path):
                
                print('path_exists: ', consensus_sample_path)
                mbs_consensus_name = mbs_consensus_names[virus]
                mbs_consensus = SeqIO.to_dict(SeqIO.parse(consensus_baseline_path, "fasta"))
                sample_consensus = SeqIO.to_dict(SeqIO.parse(consensus_sample_path, "fasta"))
                
                samples_score_metrics_parameter_dict[parameter] = get_scores_for_sample(virus, sample_consensus, mbs_consensus, mbs_consensus_name, 
                                                                                        match_score, mismatch_score, open_gap_score, extend_gap_score)
            else: 
                print('path does not exist: ', consensus_sample_path)

        samples_score_metrics_dict[sample_id] = samples_score_metrics_parameter_dict
    
    return samples_score_metrics_dict


def extract_numerical_value(string):
    # Define a regular expression pattern to match numerical values
    pattern = r'\d+\.?\d*'  # This pattern matches one or more digits followed by an optional decimal point and zero or more digits

    # Use re.findall() to find all matches of the pattern in the string
    numerical_values = re.findall(pattern, string)

    # Convert the extracted numerical values from strings to floats
    numerical_values = [float(value) for value in numerical_values]

    return numerical_values


def has_numerical_value(string):
    # Define a regular expression pattern to match numerical values
    pattern = r'\d+\.?\d*'  # This pattern matches one or more digits followed by an optional decimal point and zero or more digits

    # Use re.search() to search for the pattern in the string
    match = re.search(pattern, string)

    # If a match is found, return True; otherwise, return False
    if match:
        return True
    else:
        return False


def convert_score_metrics_json_to_df(samples_score_metrics_parameter_dict): 

    scores_colums = ['parameters', 'score', 'match', 'gap', 'mismatch', 'N_count', 'score_agg']
    samples_score_metrics_parameter_df = pd.DataFrame(columns=scores_colums)

    # sample_name = [samples_names[virus]]
    for parameter in list(samples_score_metrics_parameter_dict.keys()):  # samples_score_metrics_dict[sample]
        score =  samples_score_metrics_parameter_dict[parameter]['all_fragments']['score']
        match = samples_score_metrics_parameter_dict[parameter]['all_fragments']['matches']
        gap = samples_score_metrics_parameter_dict[parameter]['all_fragments']['gaps']
        mismatch = samples_score_metrics_parameter_dict[parameter]['all_fragments']['mismatches']
        n_count = samples_score_metrics_parameter_dict[parameter]['all_fragments']['N_count']
        score_agg = score - gap - mismatch

        params_df = pd.DataFrame([[parameter, score, match, gap, mismatch, n_count, score_agg]], columns=scores_colums)
        samples_score_metrics_parameter_df = pd.concat([samples_score_metrics_parameter_df, params_df], axis=0, ignore_index=True)

    col_names_param = ['trimmer', 'variant_caller', 'param_name', 'score', 'match', 'gap', 'mismatch', 'N_count', 'score_agg']
    scores_df = pd.DataFrame(columns=col_names_param)

    for _, row in samples_score_metrics_parameter_df.iterrows():
        
        row_scores_df = pd.DataFrame([[np.nan, np.nan, row['parameters'], row['score'], row['match'], row['gap'], row['mismatch'], 
                                    row['N_count'], row['score_agg']]], columns=col_names_param)
        param_name = row['parameters']
        print('param_name: ', param_name)
        
        param_name_split = param_name.split('_')
        print('param_name_split: ', param_name_split)
        
        row_scores_df['trimmer'] = param_name_split[0]
        
        row_scores_df['variant_caller'] = param_name_split[1]
        
        scores_df = pd.concat([scores_df, row_scores_df], axis=0, ignore_index=True)
    
    return scores_df

cols_scores = ['match_score', 'mismatch_score', 'open_gap_score', 'extend_gap_score']
set_scores_PairwiseAligner_df = pd.DataFrame([#[1, -1, 0, 0], 
                                              [1, -1, -1, -1]
                                            #   [1, -1, -2, -2]
                                              #[1, -1, -5, -1], 
                                              #[2, -1, -5, -1]
                                              ], 
                                              columns=cols_scores)

x = datetime.datetime.now()
d = str(x.year) + '_' + str(x.month) + '_' + str(x.day)
# d = '2024_3_27'
# variant_callers = ['bcftools', 'freebayes', 'ivar']
# variant_caller = variant_callers[0]

# for variant_caller in variant_callers: 
for virus in virus_reference.keys():
    print(virus)

    for _, row in set_scores_PairwiseAligner_df.iterrows():
        
        match_score = row['match_score']
        mismatch_score = row['mismatch_score']
        open_gap_score = row['open_gap_score']
        extend_gap_score = row['extend_gap_score']

        samples_score_metrics_dict = get_samples_score_metrics_dict(virus, match_score,
                                                                    mismatch_score, 
                                                                    open_gap_score, 
                                                                    extend_gap_score
                                                                    )
        print('Got metrics score dict')

        scores_df = convert_score_metrics_json_to_df(samples_score_metrics_dict[samples_names[virus]])
        scores_df = scores_df.sort_values(by=['variant_caller', 'score'], ascending=[True, False])
        
        print('dict with metrics scores converdted to df')

        path_to_save = './data/result_metrics/' + virus + '/' + virus + '_' + samples_names[virus].split('-')[-1] \
                        + 'match' + str(match_score) + '_mis' + str(mismatch_score) + '_o' + str(open_gap_score) + '_e' + str(extend_gap_score) + '_' + d + '_trimm_vc.tsv'

        scores_df.to_csv(path_to_save, sep='\t', index=False)
        print('scores saved to: ', path_to_save)
    


h1n1
run230320_UVZ_BA_23-vsp-1337_
path_exists:  ./data/report_pollux/run230320_UVZ_BA_23-vsp-1337_trimmomatic_bcftools/_summary/consensus-h1n1_2019-wgs.fa
sample_consensus:  dict_keys(['run230320_UVZ_BA_23-vsp-1337_trimmomatic_bcftools', 'A__WISCONSIN__588__2019__H1N1_PB1', 'A__WISCONSIN__588__2019__H1N1_PA', 'A__WISCONSIN__588__2019__H1N1_HA', 'A__WISCONSIN__588__2019__H1N1_NP', 'A__WISCONSIN__588__2019__H1N1_NA', 'A__WISCONSIN__588__2019__H1N1_MP', 'A__WISCONSIN__588__2019__H1N1_NS'])
sample_fragment:  run230320_UVZ_BA_23-vsp-1337_trimmomatic_bcftools
fragment:  bcftools
sample_consensus[sample_fragment]:  ID: run230320_UVZ_BA_23-vsp-1337_trimmomatic_bcftools
Name: run230320_UVZ_BA_23-vsp-1337_trimmomatic_bcftools
Description: run230320_UVZ_BA_23-vsp-1337_trimmomatic_bcftools
Number of features: 0
Seq('TCAAATATATTCAATATGGAGAGAATAAAAGAGCTGAGAGATCTAATGTCGCAG...CGA')
mbs_consensus_fragment run230320_UVZ_BA_23-vsp-1337_A__WISCONSIN__588__2019__H1N1_PB2
ID: run230320_UVZ_BA_23-vsp-1337_A

In [16]:
scores_df

trimmer variant_caller             param_name    score  match gap  \
0  trimmomatic       bcftools   trimmomatic_bcftools  14801.0  15015  12   
1     cutadapt       bcftools      cutadapt_bcftools  14801.0  15015  12   
2  trimmomatic      freebayes  trimmomatic_freebayes  14801.0  15015  12   
3     cutadapt      freebayes     cutadapt_freebayes  14801.0  15015  12   
4  trimmomatic           ivar       trimmomatic_ivar  15221.0  15221   0   
5     cutadapt           ivar          cutadapt_ivar  15221.0  15221   0   

  mismatch N_count  score_agg  
0      202       0    14587.0  
1      202       0    14587.0  
2      202       0    14587.0  
3      202       0    14587.0  
4        0       0    15221.0  
5        0       0    15221.0